In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np

from unlearning.tool import get_hf_model
from unlearning.feature_activation import get_forget_retain_data, tokenize_dataset, get_feature_activation_sparsity, get_top_features
from unlearning.jump_relu import load_gemma2_2b_sae
from unlearning.intervention import scaling_intervention
from unlearning.metrics import calculate_MCQ_metrics

In [2]:
model = get_hf_model('google/gemma-2-2b-it')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [3]:
forget_dataset, retain_dataset = get_forget_retain_data('bio-forget-corpus', 'wikitext')

print(len(forget_dataset), len(forget_dataset[0]))
print(len(retain_dataset), len(retain_dataset[0]))

forget_tokens = tokenize_dataset(model, forget_dataset)
retain_tokens = tokenize_dataset(model, retain_dataset)

print(forget_tokens.shape, retain_tokens.shape)

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

24432 16027
1962 859


In [ ]:
layer = 3
sae = load_gemma2_2b_sae(layer=layer)

# shuffle forget_tokens 
shuffled_forget_tokens = forget_tokens[torch.randperm(forget_tokens.shape[0])]

mean_feature_activation_forget = get_feature_activation_sparsity(model, sae, shuffled_forget_tokens[:2048], batch_size=8)
mean_feature_activation_retain = get_feature_activation_sparsity(model, sae, retain_tokens, batch_size=8)

# top_features = get_top_features(mean_feature_activation_forget, mean_feature_activation_retain, 0.01)
# np.savetxt(f'../data/top_features/gemma-2-2b-it-sparsity/layer{layer}.txt', top_features, fmt='%d')